## 드라이브 불러오기

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

## 이미지 나누기 (train, val, test)

In [1]:
!pip install split-folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders   # 입력된 비율을 기준으로 데이터셋을 분할하는 함수
input_folder = "/content/drive/MyDrive/Capstone/모델링/이미지"
output = "/content/drive/MyDrive/Capstone/모델링/splited_image_LEE" # where you want the split datasets saved. one will be created if it does not exist or none is set

splitfolders.ratio(input_folder, output=output, seed=777, ratio=(.8, .1, .1)) # train,test를 8:2 split 후, train,val을 9:1로 split

## 이미지 파일이 아닌 경우 삭제 (오류 방지)

In [ ]:
import os
from PIL import Image

dir_path = "/content/drive/MyDrive/Capstone/모델링/splited_image_LEE"

# 모든 하위 폴더의 파일 탐색
for root, dirs, files in os.walk(dir_path):
    for file in files:
        try:
            # 이미지 파일인 경우에만 처리
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                # 이미지 열기
                image_path = os.path.join(root, file)
                img = Image.open(image_path)
                img.verify()
        except (IOError, SyntaxError) as e:
            # 손상된 이미지 파일 삭제
            print(f"Removing corrupted file: {image_path}")
            os.remove(image_path)

## 모델링 (VGG16)

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

# 데이터 경로 설정
file_path = "/content/drive/MyDrive/Capstone/모델링/splited_image_LEE"
train_dir = os.path.join(file_path, 'train')
val_dir = os.path.join(file_path, 'val')
test_dir = os.path.join(file_path, 'test')

# 이미지 데이터 전처리
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   rotation_range=20, 
                                   width_shift_range=0.1, 
                                   height_shift_range=0.1, 
                                   shear_range=0.1, 
                                   zoom_range=0.1, 
                                   horizontal_flip=True, 
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(224, 224), 
                                                    batch_size=32, 
                                                    class_mode='categorical')
val_generator = val_datagen.flow_from_directory(val_dir, 
                                                target_size=(224, 224), 
                                                batch_size=32, 
                                                class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, 
                                                   target_size=(224, 224), 
                                                   batch_size=32, 
                                                   class_mode='categorical')

# VGG16 모델 로드하기
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 새로운 분류 레이어 추가하기
x = vgg_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# 새로운 모델 구성하기
model = Model(inputs=vgg_model.input, outputs=predictions)

# 미세 조정(fine-tuning)을 위한 가중치 동결하기
for layer in vgg_model.layers:
    layer.trainable = False

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 학습
history = model.fit(train_generator, 
                    epochs=10, 
                    validation_data=val_generator)

# 모델 평가
loss, accuracy = model.evaluate(test_generator)
print("Test accuracy:", accuracy)

# 모델 저장
model.save('/content/drive/MyDrive/Capstone/모델링/leecnn.h5')

## 모델 불러오기

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model

import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions

from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/Capstone/모델링/leecnn.h5')

## 예측

In [ ]:
img_path = '/content/drive/MyDrive/Capstone/모델링/LEE_one/predict/만리장성2.jpg' # 예측하고자 하는 이미지 경로
target_size = (224, 224) # 모델이 학습시킨 이미지 사이즈와 동일하게 설정

# 이미지 전처리
img = load_img(img_path, target_size=target_size)
x = img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0 # 이미지 스케일링

# 예측
preds = model.predict(x)

# 클래스별 확률 값 출력
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

for i in range(len(class_names)):
  print(f"{class_names[i]}: {round(preds[0][i]*100, 2)}%")